In [1]:
import numpy as np
import pandas as pd

from random import choice

from bs4 import BeautifulSoup
import requests
import re
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from IPython.display import clear_output

In [2]:
## Loading review data
br = pd.read_csv('BookReviews.csv').dropna()
br = br.reset_index(drop=True)

## creating numpy array of review data with only unique user href's
br_u = br['User Href'].unique()
len(br_u)

59881

In [3]:
## Defining function to continuing spamming requests until html can be processed
## when looking to iterate through multiple links while scraping

# returns a soup and request
# 
# The caveat is that by the definition, the function must find some object 
# that exists on the page in order to work 

def spamRequestsFind(link, tag, attr, attr_id):
    status = None
    while status == None:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, "lxml")
        status = type(soup.find(tag, attrs = {attr : attr_id}))

    prefix = len('https://www.goodreads.com/user/show/')
    userID_disp = link[prefix:]
    match = re.match(r'^(\d+)-', userID_disp)
    if match:
        userID = match.group(1)
    return(soup, r)

In [4]:
def updated_url_html(r):
    # Set up the Selenium driver with the path to the ChromeDriver executable
    options = Options()
    options.add_argument('--headless')
    service = Service(r"C:\Users\marty\OneDrive - The George Washington University\Documents\CSCI 4443\Project\chromedriver_win32\chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=options)

    # Load the Goodreads author webpage
    url = r.url
    driver.get(url)

    # Get the page source and parse it with BeautifulSoup
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # Close the Selenium driver
    driver.quit()

    return(soup)
        

In [17]:
itr = pd.read_csv("userInfo.csv")
start = len(itr)
end = start + 50

linklst = br_u[start:end]


def getUserInformation(linklst):
    while True: 
        try:
            # intializing tag to search for with spamRequestsFind to verify webpage html was successfully retrieved
            # tag is chosen as something common to any goodreads page, the siteHeader
            tag = 'div'
            attr = 'class'
            attr_id = 'siteHeader'

            # creating empty lists to house scraped information
            user_ID = []
            Display_Name = []
            num_ratings = []
            avg_rating = []
            num_reviews = []
            booklst = []
            isAuthor = []

            for i, x in enumerate(linklst):

                if i % 10 == 0:
                    clear_output()

                soup, r = spamRequestsFind(x, tag, attr, attr_id)

                user_ID += [re.sub(r'\D', '', x)]

                # executing scrape for goodreads author page
                if soup.find('h3', attrs = {'class' : 'right goodreadsAuthor'}) != None:
                    print(i, 'author')

                    Display_Name += [soup.find('h1', attrs = {'class' : 'authorName'}).text.strip()]

                    userInfo = soup.find('div', attrs = {'class' : 'smallText'}).find_all('a')
                    num_ratings += [re.sub(r'[^\d.]', '', userInfo[0].text)]
                    avg_rating += [re.sub(r'[^\d.]', '', userInfo[2].text)]
                    num_reviews += [re.sub(r'[^\d.]', '', userInfo[1].text)]
                    booklst += [None]
                    isAuthor += ['yes']
                
                # executing scrape for user whose href has changed after becoming a goodreads verified author
                elif r.url != x and soup.find('h3', attrs = {'class' : 'right goodreadsAuthor'}) != None:
                    print(i, 'normal user to author')

                    soup = updated_url_html(r)

                    Display_Name += [soup.find('h1', attrs = {'class' : 'authorName'}).text.strip()]

                    userInfo = soup.find('div', attrs = {'class' : 'smallText'}).find_all('a')
                    num_ratings += [re.sub(r'[^\d.]', '', userInfo[0].text)]
                    avg_rating += [re.sub(r'[^\d.]', '', userInfo[2].text)]
                    num_reviews += [re.sub(r'[^\d.]', '', userInfo[1].text)]
                    booklst += [None]
                    isAuthor += ['yes']

                # executing scrape for normal user account
                elif soup.find('h1', attrs = {'class' : 'userProfileName'}) != None:
                    print(i, 'normal user')

                    # Retrieving the name of the user
                    Display_Name += [soup.find('h1', attrs = {'class' : 'userProfileName'}).text.strip()]

                    # Retrieving userInfo element that houses a tags with information of user total reviews and ratings 
                    userInfo = soup.find('div', attrs = {'class' : 'profilePageUserStatsInfo'}).find_all('a')
                    num_ratings += [re.sub(r'[^\d.]', '', userInfo[0].text)]
                    avg_rating += [re.sub(r'[^\d.]', '', userInfo[1].text)]
                    num_reviews += [re.sub(r'[^\d.]', '', userInfo[2].text.strip())]

                    # Retrieving up to 10 of users favorite books
                    if soup.find('div', attrs = {'class' : 'imgGrid'}) != None:
                        favBooks = soup.find('div', attrs = {'class' : 'imgGrid'}).find_all('img')
                        books = []
                        for i,x in enumerate(favBooks):
                            books += [favBooks[i]['title']]
                        booklst += [books]
                    else:
                        booklst += [None]
                    
                    isAuthor += ['no']

                # executing for private user who has changed their display name
                elif r.url != x and soup.find('div', attrs = {'class' : 'mainContentFloat'}) != None:
                    print(i, 'private, name change')

                    soup = updated_url_html(r)

                    # Retrieving the name of the user
                    Display_Name += [soup.find('div', attrs = {'class' : 'mainContentFloat'}).find('h1').text.strip()]

                    # Retrieving userInfo element that houses a tags with information of user total reviews and ratings 
                    userInfo = soup.find('div', attrs = {'class' : 'smallText'}).find_all('a')
                    avg_rating += [re.sub(r'[^\d.]', '', userInfo[0].text)]

                    # text order is 'total_ratings|total_reviews:avg_rating'
                    cleaner_info = re.sub(r'[^0-9:|]', '', soup.find('div', attrs = {'class' : 'smallText'}).text.strip().replace('\n', '').replace('\t', ''))
                    num_ratings += [cleaner_info.split('|',1)[0]]
                    num_reviews += [re.findall(r'\|(.*?)\:', cleaner_info)[0]]

                    # Cannot see favorited books due to privacy settings
                    booklst += [None]

                    isAuthor += ['no']

                # executing scrape for private user account
                else:
                    print(i, 'private user')

                    # Retrieving the name of the user
                    Display_Name += [soup.find('div', attrs = {'class' : 'mainContentFloat'}).find('h1').text.strip()]

                    # Retrieving userInfo element that houses a tags with information of user total reviews and ratings 
                    userInfo = soup.find('div', attrs = {'class' : 'smallText'}).find_all('a')
                    avg_rating += [re.sub(r'[^\d.]', '', userInfo[0].text)]

                    # text order is 'total_ratings|total_reviews:avg_rating'
                    cleaner_info = re.sub(r'[^0-9:|]', '', soup.find('div', attrs = {'class' : 'smallText'}).text.strip().replace('\n', '').replace('\t', ''))
                    num_ratings += [cleaner_info.split('|',1)[0]]
                    num_reviews += [re.findall(r'\|(.*?)\:', cleaner_info)[0]]

                    # Cannot see favorited books due to privacy settings
                    booklst += [None]

                    isAuthor += ['no']

        except (AttributeError):
            continue
        else:
            break
    

    reviewData = pd.DataFrame({ 'User ID' : user_ID,
                                'Display Name' : Display_Name,
                                'Average Rating' : avg_rating,
                                'Total Ratings' : num_ratings,
                                'Total Reviews' : num_reviews,
                                'Favorite Books' : booklst,   
                                'Author' : isAuthor            })


    return(reviewData)

revData = getUserInformation(linklst)
revData.to_csv("userInfo.csv", index=False, mode="a", header=False)

40 normal user
41 author
42 author
43 normal user
44 normal user
45 normal user
46 private user
47 private user
48 normal user
49 normal user


In [20]:
itr = pd.read_csv("userInfo.csv")
itr[start:end]

175